In [0]:
%python
## Import libraries
import pyspark as ps
from pyspark.sql.functions import *
from pyspark.sql.types import *
from datetime import *
import pandas as pd
import json

In [0]:
class ValidateOMNI:
  def __init__(self):
    self.query = "Query About to fill"
    self.sql_df = pd.DataFrame()
    self.data_json = {"Data":[]}    
  
  def logger(message):
    print(message)
    
  def set_query_string(self):
    query = """\
    WITH t(usecase_id, hist_retention) AS (SELECT USECASE_ID,hist_retention from com_us_alyt_ngebox.metadata_usecase)
update com_us_alyt_ngebox.NGEBox_Suggestion_Requests_Processed process_table set process_table.Not_Sent_To_Veeva_Reason = (case  when ISNULL(process_table.Not_Sent_To_Veeva_Reason)= true then 'Invalid OMNI_ID Value' else concat(process_table.Not_Sent_To_Veeva_Reason, '|', 'Invalid OMNI_ID Value' ) end)
where HCP_OMNI_ID  not in (select xref.omni_id from com_us_hub.ref_cust_xref xref where xref.src_id = account_vod__c and  xref.src_system ='iEngage')
and created_date >= date_add(current_date, -(select t.hist_retention from t where t.usecase_id = usecase_id));"""
    # and SUGGEST_EXTERNAL_ID_VOD__C in (select SUGGESTION_EXTERNAL_ID_VOD__C from com_us_lake.rep_suggestion_vod__c where DISMISSED_VOD__C = 0) removed for dev
    self.query = query    

  def run_query_and_set_sqldf(self):
    query_list = self.query.split(";")
    self.sql_df = []
    for curr_query in query_list:
      if curr_query.strip() == "":
        continue
      sql_df = sqlContext.sql(curr_query).collect() # type(sql_df) -> dict
      self.sql_df = self.sql_df + sql_df
    
  def format_sqldf_to_json(self):
    sql_output_rowlist = []
    for curr_row in self.sql_df: 
      sql_output_rowlist.append (curr_row.asDict(True))
    sql_df_new = pd.DataFrame(sql_output_rowlist)
    sql_jsonstr = sql_df_new.to_json(orient="records") # type(result) -> str
    sql_jsonparse = json.loads(sql_jsonstr) # type(parsed_json) -> list
    nb_json = {"data":sql_jsonparse} # type(metadata_json) -> dict
    self.nb_json = nb_json
    
  def construct_microservice_response(self):
    resp = {
      "nge_response": {
        "status": 200,
        "body": self.nb_json
      }
    }
    self.response = resp

In [0]:
nb_obj = ValidateOMNI()
nb_obj.set_query_string()
nb_obj.run_query_and_set_sqldf()
nb_obj.format_sqldf_to_json()
nb_obj.construct_microservice_response()
dbutils.notebook.exit(
  nb_obj.response
)

{'nge_response': {'status': 200, 'body': {'data': [{'num_affected_rows': 1}]}}}